### setup

In [ ]:
import requests
import json

url_base = "https://sdt-validation-server.urban.org/api"
#url_base = "http://127.0.0.1:8000/api"
url_stub = f"{url_base}" 

/users/login/

In [ ]:
engine_account = {
    "email": "engine@example.com",  # engine username
    "password": ""   # engine password
}

user_account = {
    "email": "",
    "password": ""
}

r = requests.post(f"{url_base}/users/login/", data=engine_account)
r2 = requests.post(f"{url_base}/users/login/", data=user_account)


In [ ]:
print(r.status_code)
print(r.reason)
print(r.text)

engine_token = r.json()["token"]
print(engine_token)

engine_headers = {"Authorization": f"Token {engine_token}"}

print(r2.status_code)
print(r2.reason)
print(r2.text)

user_token = r2.json()["token"]
print(user_token)

headers = {"Authorization": f"Token {user_token}"}


### /api/job/

#### submit new job

In [ ]:
#file = {'script': ('R-scripts/puf-table.R')}
files = {'script': open('R-scripts/puf-table.R', 'rb')}

payload = {
    "title": "Example job",
    "dataset_id": "puf_2012",
}
print(url_stub)
r = requests.post(f"{url_stub}/job/jobs/", headers=headers, files=files, data=payload)

In [ ]:
import uuid
print(r.status_code)
print(r.reason)
print(r.text)


for key, value in r.json().items():
    print(key, value)

job_id_str = r.json()["id"]
job_id = uuid.UUID(r.json()["id"])

#### list all jobs

In [ ]:
r = requests.get(f"{url_stub}/job/jobs/", headers=headers)
print(r.status_code)
print(r.reason)
print(r.text)

for item in r.json():
    print(item)

#### retrieve particular job

In [ ]:
print(job_id)
r = requests.get(f"{url_stub}/job/jobs/{job_id}/", headers=headers)
print(r.status_code)
print(r.reason)
print(r.text)

for key, value in r.json().items():
    print(key, value)


#### engine posts status update

In [ ]:

run_id = 1
print(job_id)
payload = {
    "status": {'ok': True, 'info': 'completed', 'errormsg': None}
}
payload_json = json.dumps(payload)
headers["Content-Type"] = "application/json"

r = requests.patch(f"{url_stub}/job/jobs/{job_id}/runs/{run_id}/", headers=headers, data=payload_json)

In [ ]:
print(r.status_code)
print(r.reason)
for key, value in r.json().items():
    print(key, value)

#### check status of run

In [ ]:
run_id = 1
r = requests.get(f"{url_stub}/job/jobs/{job_id}/runs/{run_id}", headers=headers)

print(r.status_code)
print(r.reason)
for key, value in r.json().items():
    print(key, value)

#### retrieve runs for job


In [ ]:
r = requests.get(f"{url_stub}/job/jobs/{job_id}/runs/", headers=headers)
print(r.status_code)
print(r.reason)

for item in r.json():
    print(item)

#### retrieve results for run

In [ ]:
run_id = 2
r = requests.get(f"{url_stub}/job/jobs/{job_id}/runs/{run_id}/get-csv-results/", headers=headers)
print(r.status_code)
print(r.reason)
print(r.text)

#### post a refinement

In [ ]:
payload = {
    "refined": [{"statistic_id": 6, "epsilon": 1},
    {"statistic_id": 7, "epsilon": 2},]
}
run_id=1
payload = json.dumps(payload).encode()
headers["Content-Type"] = "application/json"
r = requests.post(f"{url_stub}/job/jobs/{job_id}/runs/{run_id}/refine/", headers=headers, data=payload)

print(r.status_code)
print(r.reason)
for key, value in r.json().items():
    print(key, value)

#### release statistics

In [ ]:
payload = {
    "released": [{"statistic_id": 6, "epsilon": 1},
    {"statistic_id": 7, "epsilon": 2},]
}
payload = json.dumps(payload).encode()
headers["Content-Type"] = "application/json"
r = requests.post(f"{url_stub}/job/jobs/{job_id}/runs/{run_id}/release/", headers=headers, data=payload)

print(r.status_code)
print(r.reason)


### /api/budget

#### retrieve current budget of user

In [ ]:
r = requests.get(f"{url_stub}/budget/budget/", headers=headers)

print(r.status_code)
print(r.reason)
print(r.text)

